# 评估
评估是对应用程序的输出进行质量检查的过程。由于LLM的输出具有不可预测性和多变性，判断输出是否正常比正常程序要困难。

In [21]:
import json
from typing import List, Dict, Any, Optional
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain.evaluation import load_evaluator, EvaluatorType
from langchain.evaluation.qa import QAEvalChain
from langchain_core.language_models import FakeListLLM
from dashscope import Generation as DashGeneration

# 读取配置文件
with open("config.json", "r") as f:
    config = json.load(f)

# 创建与模型交互的函数
def chat_with_qwen(prompt):
    """简单的函数，用于与DashScope API交互"""
    if hasattr(prompt, "to_string"):
        prompt_str = prompt.to_string()
    else:
        prompt_str = str(prompt)
        
    response = DashGeneration.call(
        model=config['qwen_model'],
        prompt=prompt_str,
        api_key=config['api_key']
    )
    
    if response.status_code == 200:
        return response.output.text
    else:
        raise Exception(f"API调用失败: {response.code} - {response.message}")

# 将函数包装为Runnable对象
qwen_chain = RunnableLambda(chat_with_qwen)

# 准备评估数据
eval_data = [
    {
        "question": "什么是机器学习？",
        "ground_truth": "机器学习是人工智能的一个分支，它使用数据和算法来模仿人类学习的方式，逐步提高准确性。",
        "context": "机器学习是人工智能的一个重要分支，它通过算法让计算机从数据中学习，不断改进性能。",
    },
    {
        "question": "Python的主要特点是什么？",
        "ground_truth": "Python的主要特点包括简洁的语法、丰富的库、跨平台性和易学易用性。",
        "context": "Python是一种高级编程语言，以其清晰的语法和丰富的生态系统而闻名。它支持多种编程范式。",
    }
]

# 创建问答链
qa_prompt = PromptTemplate.from_template("""
基于以下上下文回答问题：

上下文：{context}

问题：{question}

回答：
""")

qa_chain = (
    qa_prompt 
    | qwen_chain 
    | StrOutputParser()
)

# 创建QA评估链
def create_qa_eval_chain():
    # 创建QA评估提示模板
    qa_eval_prompt = """
作为一位专业的评估专家，请评价以下问答对的质量：

问题: {query}
参考答案: {answer}
模型回答: {result}

请考虑以下几个方面：
1. 准确性：模型回答中的信息是否与参考答案一致？
2. 完整性：模型回答是否涵盖了参考答案中的关键信息？
3. 相关性：模型回答是否直接回应了问题？

评分:
- 1分: 完全不正确或无关
- 2分: 大部分不正确
- 3分: 部分正确
- 4分: 大部分正确
- 5分: 完全正确

请给出1-5分的评分并解释原因。

评估:
"""

    # 创建QA评估链
    eval_chain = QAEvalChain.from_llm(
        llm=qwen_chain,
        prompt=PromptTemplate.from_template(qa_eval_prompt)
    )
    
    return eval_chain

# 使用字符串距离评估器
def get_string_distance_evaluator():
    return load_evaluator(EvaluatorType.STRING_DISTANCE)

# 评估函数
def evaluate_qa_system(eval_data: List[Dict[str, Any]]):
    # 创建评估器
    qa_eval_chain = create_qa_eval_chain()
    string_evaluator = get_string_distance_evaluator()
    
    results = []
    predictions = []
    
    print("生成模型回答...")
    # 首先，获取所有问题的模型回答
    for i, item in enumerate(eval_data):
        try:
            # 获取模型回答
            prediction = qa_chain.invoke({
                "context": item["context"],
                "question": item["question"]
            })
            
            # 为QAEvalChain准备数据
            predictions.append({
                "query": item["question"],
                "answer": item["ground_truth"],
                "result": prediction
            })
            
            # 基本评估结果
            eval_result = {
                "question": item["question"],
                "prediction": prediction,
                "ground_truth": item["ground_truth"],
                "evaluations": {}
            }
            
            # 执行字符串距离评估
            string_eval = string_evaluator.evaluate_strings(
                prediction=prediction,
                reference=item["ground_truth"]
            )
            eval_result["evaluations"]["string_distance"] = string_eval["score"]
            
            results.append(eval_result)
            print(f"  问题 {i+1}: 已生成回答")
            
        except Exception as e:
            print(f"  问题 {i+1} 生成失败: {str(e)}")
            continue
    
    # 如果没有成功生成任何回答，直接返回
    if not predictions:
        return results
    
    print("使用QAEvalChain进行评估...")
    try:
        # 使用QAEvalChain进行评估
        qa_evals = qa_eval_chain.evaluate(
            predictions,
            predictions
            )
        
        # 提取QA评估分数并添加到结果中
        for i, eval_result in enumerate(results):
            if i < len(qa_evals):
                qa_eval = qa_evals[i]
                # 提取评分
                score, explanation = extract_score_from_qa_eval(qa_eval.get("text", ""))
                
                eval_result["evaluations"]["qa_eval"] = {
                    "score": score,
                    "explanation": explanation,
                    "raw": qa_eval.get("text", "")
                }
                
                print(f"  问题 {i+1}: 评分={score}")
            
    except Exception as e:
        print(f"QA评估失败: {str(e)}")
        import traceback
        traceback.print_exc()
    
    return results

# 从QA评估文本中提取分数
def extract_score_from_qa_eval(text):
    import re
    
    # 尝试匹配数字评分
    score_match = re.search(r'(\d+)分', text)
    if score_match:
        try:
            score = int(score_match.group(1))
            if 1 <= score <= 5:
                # 提取解释（评分后的文本）
                explanation = text[score_match.end():].strip()
                return score, explanation
        except:
            pass
    
    # 如果没有找到明确的分数，尝试根据关键词判断
    if "完全正确" in text or "非常好" in text:
        return 5, text
    elif "大部分正确" in text:
        return 4, text
    elif "部分正确" in text:
        return 3, text
    elif "大部分不正确" in text:
        return 2, text
    elif "完全不正确" in text or "无关" in text:
        return 1, text
    
    # 如果无法确定分数，返回一个中等分数
    return 3, text

# 结果分析函数
def analyze_results(results: List[Dict[str, Any]]):
    if not results:
        return {
            "total_questions": 0,
            "error": "没有可用的评估结果"
        }
        
    summary = {
        "total_questions": len(results),
        "average_scores": {
            "string_distance": 0,
            "qa_eval": 0
        },
        "detailed_results": results
    }
    
    qa_eval_count = 0
    
    for result in results:
        evaluations = result.get("evaluations", {})
        
        # 字符串距离
        if "string_distance" in evaluations:
            summary["average_scores"]["string_distance"] += evaluations["string_distance"]
        
        # QA评估
        if "qa_eval" in evaluations and "score" in evaluations["qa_eval"]:
            summary["average_scores"]["qa_eval"] += evaluations["qa_eval"]["score"]
            qa_eval_count += 1
    
    # 计算平均值
    total = len(results)
    if total > 0:
        summary["average_scores"]["string_distance"] /= total
        
        if qa_eval_count > 0:
            summary["average_scores"]["qa_eval"] /= qa_eval_count
        else:
            # 如果没有QA评估分数，从结果中移除
            summary["average_scores"].pop("qa_eval", None)
    
    return summary

# 主评估函数
def run_evaluation():
    print("开始评估...")
    
    try:
        # 运行评估
        results = evaluate_qa_system(eval_data)
        
        # 分析结果
        summary = analyze_results(results)
        
        # 打印结果
        print("\n评估结果摘要:")
        print(f"总评估问题数: {summary['total_questions']}")
        
        if summary['total_questions'] > 0:
            print("\n平均分数:")
            for metric, score in summary["average_scores"].items():
                print(f"{metric}: {score:.2f}")
            
            print("\n详细评估结果:")
            for i, result in enumerate(summary["detailed_results"]):
                print(f"\n问题 {i+1}: {result['question']}")
                print(f"模型回答: {result['prediction']}")
                print(f"参考答案: {result['ground_truth']}")
                
                print("评估分数:")
                if "string_distance" in result["evaluations"]:
                    print(f"- 字符串距离: {result['evaluations']['string_distance']:.2f}")
                
                if "qa_eval" in result["evaluations"]:
                    print(f"- QA评分: {result['evaluations']['qa_eval']['score']}")
                    print(f"- 评估解释: {result['evaluations']['qa_eval']['explanation']}")
                    
        else:
            print("\n没有成功的评估结果")
            
    except Exception as e:
        print(f"评估过程中出现错误: {str(e)}")
        import traceback
        traceback.print_exc()

# 运行评估
run_evaluation()

开始评估...
生成模型回答...
  问题 1: 已生成回答
  问题 2: 已生成回答
使用QAEvalChain进行评估...
QA评估失败: chat_with_qwen() got an unexpected keyword argument 'stop'

评估结果摘要:
总评估问题数: 2

平均分数:
string_distance: 0.27

详细评估结果:

问题 1: 什么是机器学习？
模型回答: 机器学习是人工智能的一个重要分支，它通过算法让计算机从数据中学习，不断改进性能。
参考答案: 机器学习是人工智能的一个分支，它使用数据和算法来模仿人类学习的方式，逐步提高准确性。
评估分数:
- 字符串距离: 0.17

问题 2: Python的主要特点是什么？
模型回答: Python的主要特点是其清晰的语法和丰富的生态系统，同时它支持多种编程范式。
参考答案: Python的主要特点包括简洁的语法、丰富的库、跨平台性和易学易用性。
评估分数:
- 字符串距离: 0.36


Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_41744\1658730350.py", line 162, in evaluate_qa_system
    qa_evals = qa_eval_chain.evaluate(
               ^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anaconda\Lib\site-packages\langchain\evaluation\qa\eval_chain.py", line 147, in evaluate
    return self.apply(inputs, callbacks=callbacks)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anaconda\Lib\site-packages\langchain\chains\llm.py", line 251, in apply
    raise e
  File "d:\Anaconda\Lib\site-packages\langchain\chains\llm.py", line 248, in apply
    response = self.generate(input_list, run_manager=run_manager)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anaconda\Lib\site-packages\langchain\chains\llm.py", line 145, in generate
    results = self.llm.bind(stop=stop, **self.llm_kwargs).batch(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "d:\Anaconda\Lib\site-packages\langcha